# DSC540-T301 Data Preparation (2203-1)
## Final Project
### Ambrose M Malagon Cordero
________________________

*Instructions: During the course, you will be working on a term project to either pull data from an API or scrape a webpage. You will need to select either an API (different than Twitter) or a Webpage and create a process in Python that will extract data into a formatted dataset.* 

For this term project, I've chosen to use the Marvel Comics API (https://developer.marvel.com) which allows access to information about Marvel's library of comics. In order to access the API, you will need to register in order to obtain both a public and private key. 

In [1]:
# RUN THIS CELL - Otherwise the program won't work.
public_key = '385cebbcb01e23edbff1c01d0be2dca3'
private_key = '4c672a897525001a358fe052fc0ff08e036950bd'

### How the API works
Per the documentation provided by Marvel (https://developer.marvel.com/documentation/authorization), under **Authentication for Server-Side Applications**:

*Server-side applications must pass two parameters in addition to the apikey parameter:
ts - a timestamp (or other long string which can change on a request-by-request basis)
hash - a md5 digest of the ts parameter, your private key and your public key (e.g. md5(ts+privateKey+publicKey)
For example, a user with a public key of "1234" and a private key of "abcd" could construct a valid call as follows: http://gateway.marvel.com/v1/public/comics?ts=1&apikey=1234&hash=ffd275c5130566a2916217b101f26150 (the hash value is the md5 digest of 1abcd1234)*

_________________
#### Part I -A formatted dataset with at least 15-20 variables.
First we start by importing all of the necessary modules.

In [66]:
import hashlib  # Required in order to create the hash value.
import time     # To generate the the time stamp. 
import requests # To make the call to the API.
import pandas as pd # To store the retrieved data into a data frame.
import re # Use regex to extract certain elements from the data. 

________________
Starting to work with the API, I noticed that some queries would not work as intended. For example: When attempting to retrieve data using the query to get characters by name as established in the documentation, with the return response being that the hash/public ID, or another sequence of values provided was incorrect. This challenge does not show up when using character/event/comic id's. To test out the API, I am going to get specific character information by using the query as explained in the documentation provided by the Marvel Developer website. 

In [3]:
char_id = ['1009220', '1009664', '1009368', '1009189', '1009351', '1009338', '1009610', '1009562', '1009697', '1010338', '1009477', '1017111', '1009718']
response_ = []
for req_id in char_id:
    # Create the hash.
    time_stamp = str(time.time()) # Creating the timestamp and converting it to a string.
    time_stamp_byte = bytes(time_stamp, 'utf-8') # Now converting the time stamp into a string. 
    hash_ = hashlib.md5(time_stamp_byte) # Hashing the timestamp using the md5 format per Marvel's instructions.
    hash_.update(b"4c672a897525001a358fe052fc0ff08e036950bd") # Next, adding the Private Key as a bytes.
    hash_.update(b"385cebbcb01e23edbff1c01d0be2dca3") # Adding the public key to the hash as bytes.
    hash_t = hash_.hexdigest() # Finally, we translate the hash into a hexadecimal string and save it as the hash_t variable.
    hash_t
    # Next we declare the various pieces to construct the API url:
    base_url = "https://gateway.marvel.com"
    query = "/v1/public/characters/" + req_id + "?"
    query_url = base_url + query +"ts=" + time_stamp + "&apikey=" + public_key + "&hash=" + hash_t
    character_data = requests.get(query_url)
    response_.append(character_data.json()) #Append the resulting JSON data into a list

In [4]:
# Let's inspect one of the resulting JSON files by looking at the first element of the list.  
response_[0]

{'code': 200,
 'status': 'Ok',
 'copyright': '© 2020 MARVEL',
 'attributionText': 'Data provided by Marvel. © 2020 MARVEL',
 'attributionHTML': '<a href="http://marvel.com">Data provided by Marvel. © 2020 MARVEL</a>',
 'etag': '011e23e9a61d757b757d24075050b75cc547c8ac',
 'data': {'offset': 0,
  'limit': 20,
  'total': 1,
  'count': 1,
  'results': [{'id': 1009220,
    'name': 'Captain America',
    'description': "Vowing to serve his country any way he could, young Steve Rogers took the super soldier serum to become America's one-man army. Fighting for the red, white and blue for over 60 years, Captain America is the living, breathing symbol of freedom and liberty.",
    'modified': '2016-09-06T11:37:19-0400',
    'thumbnail': {'path': 'http://i.annihil.us/u/prod/marvel/i/mg/3/50/537ba56d31087',
     'extension': 'jpg'},
    'resourceURI': 'http://gateway.marvel.com/v1/public/characters/1009220',
    'comics': {'available': 2350,
     'collectionURI': 'http://gateway.marvel.com/v1/publ

________________
Looking at the response, we can see that the data we want is in the following pathway: 
>response -> data -> results

From it, we can extract the following data:
* Character ID.
* Character Name.
* Description.
* Last Modify Date for the Record.
* Comics Data, which includes:
    * Total Number of Comics the Character has appeared in.
    * Comic ID.
    * Comic Name.
* Series Data, which includes:
    * Total Number of Series the Character has appeared in.
    * Series ID.
    * Series Name.
* Stories Data, which includes:
    * Total Number of Stories the Character has appeared in.
    * Stories ID.
    * Story Name.
    * Story Type.
* Event Data, which includes:
    * Total Number of Events the Character has appeared in.
    * Event ID.
    * Event Name.
__________________

**Next Step: Creating a Dataframe.**

In [7]:
# Extract the results data and save it into a list.
char_data = [] 
for i in response_:
    char_data.append(i['data']['results'])

    marvel_ = [] 
for results in char_data:
    for data in results:
        details = []
        details.append(data['id'])
        details.append(data['name'])
        details.append(data['description'])
        details.append(data['modified'])
        details.append(data['comics']['available'])
        details.append(data['series']['available'])
        details.append(data['stories']['available'])
        details.append(data['events']['available'])
        for i in range(len(data['urls'])):
            type_ = data['urls'][i]['type']
            url_ = data['urls'][i]['url']
            details.append(f'{type_}: {url_}')
        marvel_.append(details)

In [10]:
# Before we convert this list into a dataframe, we need to check the length of each individual element. 
for i in marvel_:
    print(len(i))

11
11
11
11
11
11
11
11
11
11
11
10
11


In [11]:
# Inspecting the 12th list against the first list.
marvel_[11]

[1017111,
 'She-Hulk (HAS)',
 "She-Hulk is the Hulk's action-loving cousin. She's unbelievably strong, pilots the group's heavily armed Jump Jet, and uses a pair of Gamma Gauntlets that give her fists an added energy wallop. As the most competitive member of the team, She-Hulk never backs down from a fight, arm-wrestling match, or burping contest!  ",
 '2013-09-18T15:53:40-0400',
 0,
 1,
 1,
 0,
 'detail: http://marvel.com/characters/51/she-hulk?utm_campaign=apiRef&utm_source=385cebbcb01e23edbff1c01d0be2dca3',
 'comiclink: http://marvel.com/comics/characters/1017111/she-hulk_has?utm_campaign=apiRef&utm_source=385cebbcb01e23edbff1c01d0be2dca3']

In [13]:
marvel_[0]

[1009220,
 'Captain America',
 "Vowing to serve his country any way he could, young Steve Rogers took the super soldier serum to become America's one-man army. Fighting for the red, white and blue for over 60 years, Captain America is the living, breathing symbol of freedom and liberty.",
 '2016-09-06T11:37:19-0400',
 2350,
 643,
 3491,
 30,
 'detail: http://marvel.com/comics/characters/1009220/captain_america?utm_campaign=apiRef&utm_source=385cebbcb01e23edbff1c01d0be2dca3',
 'wiki: http://marvel.com/universe/Captain_America_(Steve_Rogers)?utm_campaign=apiRef&utm_source=385cebbcb01e23edbff1c01d0be2dca3',
 'comiclink: http://marvel.com/comics/characters/1009220/captain_america?utm_campaign=apiRef&utm_source=385cebbcb01e23edbff1c01d0be2dca3']

She_Hulk does not have wiki url. Let's take a look at the total count of urls available per character before we continue any further.

In [21]:
# Iterate over the urls to get a list of what types of url's each character has.
for results in char_data:
    for data in results:
        url_t = []
        for i in range(len(data['urls'])):
            url_t.append(data['urls'][i]['type'])
        print(data['name'],url_t)

Captain America ['detail', 'wiki', 'comiclink']
Thor ['detail', 'wiki', 'comiclink']
Iron Man ['detail', 'wiki', 'comiclink']
Black Widow ['detail', 'wiki', 'comiclink']
Hulk ['detail', 'wiki', 'comiclink']
Hawkeye ['detail', 'wiki', 'comiclink']
Spider-Man ['detail', 'wiki', 'comiclink']
Scarlet Witch ['detail', 'wiki', 'comiclink']
Vision ['detail', 'wiki', 'comiclink']
Captain Marvel (Carol Danvers) ['detail', 'wiki', 'comiclink']
Nova ['detail', 'wiki', 'comiclink']
She-Hulk (HAS) ['detail', 'comiclink']
Wolverine ['detail', 'wiki', 'comiclink']


____________
With the pre-selected data, it appears that at most, each character can have up to 3 urls. In this case, with She-Hulk missing the wiki url, we can make the following adjustment to our code.

In [32]:
marvel_ = [] 
for results in char_data:
    for data in results:
        details = []
        details.append(data['id'])
        details.append(data['name'])
        details.append(data['description'])
        details.append(data['modified'])
        details.append(data['comics']['available'])
        details.append(data['series']['available'])
        details.append(data['stories']['available'])
        details.append(data['events']['available'])
        for i in range(len(data['urls'])):
            type_ = data['urls'][i]['type']
            url_ = data['urls'][i]['url']
            details.append(f'{type_}: {url_}')
        if len(data['urls']) <= 2: # Added conditional statement, if the url dict length is less then two, then add a blank url.
            details.append(f'wiki: NULL')  
        marvel_.append(details)

In [33]:
# Taking a look at length of the resulting lists once more.
for i in marvel_:
    print(len(i))

11
11
11
11
11
11
11
11
11
11
11
11
11


In [34]:
marvel_[11]

[1017111,
 'She-Hulk (HAS)',
 "She-Hulk is the Hulk's action-loving cousin. She's unbelievably strong, pilots the group's heavily armed Jump Jet, and uses a pair of Gamma Gauntlets that give her fists an added energy wallop. As the most competitive member of the team, She-Hulk never backs down from a fight, arm-wrestling match, or burping contest!  ",
 '2013-09-18T15:53:40-0400',
 0,
 1,
 1,
 0,
 'detail: http://marvel.com/characters/51/she-hulk?utm_campaign=apiRef&utm_source=385cebbcb01e23edbff1c01d0be2dca3',
 'comiclink: http://marvel.com/comics/characters/1017111/she-hulk_has?utm_campaign=apiRef&utm_source=385cebbcb01e23edbff1c01d0be2dca3',
 'wiki: NULL']

______________
Now that the lists are of equal length, let's work on the next item - The record modify date. Let's split the date from the time.

In [38]:
for i in marvel_:
    date, time = i[3].split("T")
    i.append(date)
    i.append(time)

In [39]:
# Checking the length of the lists to ensure they still match
for i in marvel_:
    print(len(i))

13
13
13
13
13
13
13
13
13
13
13
13
13


---------------
Next, we drill into the nested data that lies within the comics and series data. 

In [200]:
comics_= []
series_= []
stories_= []
event_= []

# Iterate over the results, to then iterate over the data to drill into the respective data. 
for results in char_data:
    for data in results:
        for i in range(len(data['comics']['items'])):
            details = [] # Temp list to hold the individual records
            details.append(data['id']) # Append the character ID - We will use this later to join the data.
            details.append(data['comics']['items'][i]['resourceURI'].rpartition('/')[2]) # Extract the comic ID from the URL.
            details.append((data['comics']['items'][i]['name']))
            comics_.append(details)

for results in char_data:
    for data in results:
        for i in range(len(data['series']['items'])):
            details = []
            details.append(data['id'])
            details.append(data['series']['items'][i]['resourceURI'].rpartition('/')[2])
            details.append((data['series']['items'][i]['name']))
            series_.append(details)

for results in char_data:
    for data in results:
        for i in range(len(data['stories']['items'])):
            details = []
            details.append(data['id'])
            details.append(data['stories']['items'][i]['resourceURI'].rpartition('/')[2])
            details.append((data['stories']['items'][i]['name']))
            details.append((data['stories']['items'][i]['type']))
            stories_.append(details)

for results in char_data:
    for data in results:
        for i in range(len(data['events']['items'])):
            details = []
            details.append(data['id'])
            details.append(data['events']['items'][i]['resourceURI'].rpartition('/')[2])
            details.append((data['events']['items'][i]['name']))
            event_.append(details)

___________________
Let's take a look at the lists for each of these data to see what we can get. First, we want to extract the name, year and issue data from the comics. We will accomplish this with the use of regex. Inspecting the data, we notice that the year is encapsulated on the first set of parentheses. Though this isn't always the case as some data does not contain the year as seen below:

In [134]:
comics_[0:25]

[[1009220, '43488', 'A+X (2012) #1'],
 [1009220, '43501', 'A+X (2012) #4'],
 [1009220, '43508', 'A+X (2012) #9'],
 [1009220, '17743', 'A-Next (1998) #2'],
 [1009220, '17744', 'A-Next (1998) #3'],
 [1009220, '17745', 'A-Next (1998) #4'],
 [1009220, '17746', 'A-Next (1998) #5'],
 [1009220, '17747', 'A-Next (1998) #6'],
 [1009220, '17748', 'A-Next (1998) #7'],
 [1009220, '17749', 'A-Next (1998) #8'],
 [1009220, '17750', 'A-Next (1998) #9'],
 [1009220, '17740', 'A-Next (1998) #10'],
 [1009220, '17741', 'A-Next (1998) #11'],
 [1009220, '17742', 'A-Next (1998) #12'],
 [1009220, '66978', 'Adventures of Captain America (1991) #1'],
 [1009220, '66979', 'Adventures of Captain America (1991) #2'],
 [1009220, '66980', 'Adventures of Captain America (1991) #3'],
 [1009220, '66981', 'Adventures of Captain America (1991) #4'],
 [1009220,
  '42539',
  'Age of Apocalypse (2011) #2 (Avengers Art Appreciation Variant)'],
 [1009220, '30090', 'Age of Heroes (2010) #1'],
 [1009664, '43506', 'A+X (2012) #7']

In [205]:
# For loop to iterate over the comics_ list
for i in comics_:
    # Append the comic book name.
    name = i[2].rpartition('(')[0]
    # Try/Except block, in the event that the text does not have parenthesis with the year, then returns a Null value by default.
    i.append(name)
    try: 
        val = re.search(r'\((.*?)\)',i[2]).group(1) # Regex expression that extracts the contents on the first set of parentheses.
    # Conditional statement to check if the contents inside the parentheses are digits, if not then append NULL as the value.
        if val.isdigit():
            i.append(val)
        else:
            i.append("NULL")
    except:
        i.append("NULL")
    val2 = i[2].rpartition(' ')[-1]
    if val2[0] == '#':
        i.append(val2)
    else:
        val_r1 = re.sub('[()]', '', val2) # Remove lingering parenthesis from the any non-issue number data. 
        i.append(val_r1)

In [206]:
# A glimpse of the resulting data. 
comics_[0:10]

[[1009220, '43488', 'A+X (2012) #1', 'A+X ', '2012', '#1'],
 [1009220, '43501', 'A+X (2012) #4', 'A+X ', '2012', '#4'],
 [1009220, '43508', 'A+X (2012) #9', 'A+X ', '2012', '#9'],
 [1009220, '17743', 'A-Next (1998) #2', 'A-Next ', '1998', '#2'],
 [1009220, '17744', 'A-Next (1998) #3', 'A-Next ', '1998', '#3'],
 [1009220, '17745', 'A-Next (1998) #4', 'A-Next ', '1998', '#4'],
 [1009220, '17746', 'A-Next (1998) #5', 'A-Next ', '1998', '#5'],
 [1009220, '17747', 'A-Next (1998) #6', 'A-Next ', '1998', '#6'],
 [1009220, '17748', 'A-Next (1998) #7', 'A-Next ', '1998', '#7'],
 [1009220, '17749', 'A-Next (1998) #8', 'A-Next ', '1998', '#8']]

-------------
Next, we take a look at the series data.

In [138]:
series_[0:25]

[[1009220, '16450', 'A+X (2012 - 2014)'],
 [1009220, '3620', 'A-Next (1998 - 1999)'],
 [1009220, '24227', 'Adventures of Captain America (1991 - 1992)'],
 [1009220, '15331', 'Age of Apocalypse (2011 - 2013)'],
 [1009220, '9790', 'Age of Heroes (2010)'],
 [1009220, '10235', 'AGE OF HEROES TPB (2011)'],
 [1009220, '13896', 'Age of X: Universe (2011)'],
 [1009220, '7534', 'All Winners Comics 70th Anniversary Special (2009)'],
 [1009220, '20682', 'All-New Wolverine (2015 - 2018)'],
 [1009220, '2114', 'All-Winners Comics (1941 - 1947)'],
 [1009220, '9865', 'All-Winners Squad: Band of Heroes (2011)'],
 [1009220, '2116', 'Alpha Flight (1983 - 1994)'],
 [1009220, '454', 'Amazing Spider-Man (1999 - 2013)'],
 [1009220, '15540', 'Amazing Spider-Man Annual (2012)'],
 [1009220, '2984', 'Amazing Spider-Man Annual (1964 - 2018)'],
 [1009220, '9802', 'Amazing Spider-Man Annual (2010)'],
 [1009220, '1489', 'AMAZING SPIDER-MAN VOL. 10: NEW AVENGERS TPB (2005)'],
 [1009220, '14818', 'Annihilators: Earthf

Here we want the name and year(s) information. It looks similar to what we just did, however for the year we get either a single digit or we get a range. 

In [201]:
for i in series_:
    # Append the series name.
    name = i[2].rpartition('(')[0]
    i.append(name)
    try:
        val = re.search(r'\((.*?)\)',i[2]).group(1) # Regex expression that extracts the contents on the first set of parentheses.
    # Here we change the conditional statement to identify date ranges by looking for "-". If it does not find the "-" then it will
    # assume the value as both the start and end year of the series with a duration of 0 years. Else, it will split on "-", assigning
    # the start and end year as well as calculate the duration in years. 
        if val.find("-") == -1:
            start = int(val.strip())
            end = int(val.strip())
            duration = 0
            i.append(start)
            i.append(end)
            i.append(duration)
        else:
            start, end = val.split('-')
            start = int(start.strip())
            end = int(end.strip())
            duration = end - start
            i.append(start)
            i.append(end)
            i.append(duration)
    except:
        test.append("NULL")
        test.append("NULL")
        test.append("NULL")


In [202]:
series_[0:10]

[[1009220, '16450', 'A+X (2012 - 2014)', 'A+X ', 2012, 2014, 2],
 [1009220, '3620', 'A-Next (1998 - 1999)', 'A-Next ', 1998, 1999, 1],
 [1009220,
  '24227',
  'Adventures of Captain America (1991 - 1992)',
  'Adventures of Captain America ',
  1991,
  1992,
  1],
 [1009220,
  '15331',
  'Age of Apocalypse (2011 - 2013)',
  'Age of Apocalypse ',
  2011,
  2013,
  2],
 [1009220, '9790', 'Age of Heroes (2010)', 'Age of Heroes ', 2010, 2010, 0],
 [1009220,
  '10235',
  'AGE OF HEROES TPB (2011)',
  'AGE OF HEROES TPB ',
  2011,
  2011,
  0],
 [1009220,
  '13896',
  'Age of X: Universe (2011)',
  'Age of X: Universe ',
  2011,
  2011,
  0],
 [1009220,
  '7534',
  'All Winners Comics 70th Anniversary Special (2009)',
  'All Winners Comics 70th Anniversary Special ',
  2009,
  2009,
  0],
 [1009220,
  '20682',
  'All-New Wolverine (2015 - 2018)',
  'All-New Wolverine ',
  2015,
  2018,
  3],
 [1009220,
  '2114',
  'All-Winners Comics (1941 - 1947)',
  'All-Winners Comics ',
  1941,
  1947,
  

---------------
Now we construct our dataframes.

In [53]:
# Create the headers-data.
headers = ['char_id', 'char_name','char_description','del_mod_datetime', 'comics_total', 'series_total', 'stories_total', 'events_total',
          'url_1','url_2','url_3','record_mod_date','record_mod_time']
marvel_df = pd.DataFrame(marvel_, columns=headers) # Create the dataframe
marvel_df = marvel_df.drop('del_mod_datetime', axis=1)
marvel_df = marvel_df[['char_id', 'char_name','char_description','record_mod_date', 'record_mod_time','comics_total', 'series_total', 'stories_total', 'events_total',
          'url_1','url_2','url_3']]

In [242]:
marvel_df.head()

,char_id,char_name,char_description,record_mod_date,record_mod_time,comics_total,series_total,stories_total,events_total,url_1,url_2,url_3
0,1009220,Captain America,"Vowing to serve his country any way he could, ...",2016-09-06,11:37:19-0400,2350,643,3491,30,detail: http://marvel.com/comics/characters/10...,wiki: http://marvel.com/universe/Captain_Ameri...,comiclink: http://marvel.com/comics/characters...
1,1009664,Thor,"As the Norse God of thunder and lightning, Tho...",2019-02-06,18:10:24-0500,1749,501,2657,27,detail: http://marvel.com/characters/1009664/t...,wiki: http://marvel.com/universe/Thor_(Thor_Od...,comiclink: http://marvel.com/comics/characters...
2,1009368,Iron Man,"Wounded, captured and forced to build a weapon...",2016-09-28,12:08:19-0400,2562,627,3880,31,detail: http://marvel.com/comics/characters/10...,wiki: http://marvel.com/universe/Iron_Man_(Ant...,comiclink: http://marvel.com/comics/characters...
3,1009189,Black Widow,,2016-01-04,18:09:26-0500,546,206,628,16,detail: http://marvel.com/comics/characters/10...,wiki: http://marvel.com/universe/Black_Widow_(...,comiclink: http://marvel.com/comics/characters...
4,1009351,Hulk,Caught in a gamma bomb explosion while trying ...,2016-06-02,12:38:33-0400,1650,466,2529,26,detail: http://marvel.com/comics/characters/10...,wiki: http://marvel.com/universe/Hulk_(Bruce_B...,comiclink: http://marvel.com/comics/characters...


________
Next we move on to the comics, events, series and stories data:

In [211]:
# Create the headers
headers_comics = ['char_id', 'comic_id', 'comic_composite', 'comic_name', 'comic_year', 'comic_issue']
headers_series = ['char_id', 'series_id', 'series_composite','series_name', 'series_start_year', 'series_end_year', 'series_duration']
headers_stories = ['char_id', 'story_id', 'story_name', 'story_type']
headers_events = ['char_id', 'event_id', 'event_name']
# Create the data frames.
comics_df = pd.DataFrame(comics_, columns=headers_comics)
series_df = pd.DataFrame(series_, columns=headers_series)
stories_df = pd.DataFrame(stories_, columns=headers_stories)
events_df = pd.DataFrame(event_, columns=headers_events)
# drop the composites.
comics_df = comics_df.drop('comic_composite', axis=1)
series_df = series_df.drop('series_composite', axis=1)

Now we inspect the dataframes.

In [221]:
comics_df.head()

,char_id,comic_id,comic_name,comic_year,comic_issue
0,1009220,43488,A+X,2012,#1
1,1009220,43501,A+X,2012,#4
2,1009220,43508,A+X,2012,#9
3,1009220,17743,A-Next,1998,#2
4,1009220,17744,A-Next,1998,#3


In [214]:
series_df.head()

,char_id,series_id,series_name,series_start_year,series_end_year,series_duration
0,1009220,16450,A+X,2012.0,2014.0,2.0
1,1009220,3620,A-Next,1998.0,1999.0,1.0
2,1009220,24227,Adventures of Captain America,1991.0,1992.0,1.0
3,1009220,15331,Age of Apocalypse,2011.0,2013.0,2.0
4,1009220,9790,Age of Heroes,2010.0,2010.0,0.0


In [215]:
stories_df.head()

,char_id,story_id,story_name,story_type
0,1009220,670,X-MEN (2004) #186,cover
1,1009220,892,THOR (1998) #81,cover
2,1009220,960,3 of ?,cover
3,1009220,1026,Avengers (1998) #81,cover
4,1009220,1041,Avengers (1998) #502,interiorStory


In [241]:
events_df.head()

,char_id,event_id,event_name
0,1009220,116,Acts of Vengeance!
1,1009220,314,Age of Ultron
2,1009220,303,Age of X
3,1009220,231,Armor Wars
4,1009220,234,Avengers Disassembled


Next - We join Comics data with the Series data.


In [222]:
# Checking the data types on comics and series.
comics_df.dtypes

char_id         int64
comic_id       object
comic_name     object
comic_year     object
comic_issue    object
dtype: object

In [220]:
series_df.dtypes

char_id                int64
series_id             object
series_name           object
series_start_year    float64
series_end_year      float64
series_duration      float64
dtype: object

In [225]:
# Convert the the comic_year into a float, any errors/null values are coerced. 
comics_df["comic_year"] = pd.to_numeric(comics_df["comic_year"], errors='coerce')

In [227]:
comics_df.dtypes

char_id          int64
comic_id        object
comic_name      object
comic_year     float64
comic_issue     object
dtype: object

In [237]:
# Joining the dataframes.
comic_series = pd.merge(comics_df, series_df, left_on=  ['char_id', 'comic_name', 'comic_year'], 
                        right_on= ['char_id','series_name', 'series_start_year'], 
                        how = 'left')

In [238]:
comic_series = comic_series[['char_id', 'comic_id', 'series_id', 'comic_name', 'comic_issue', 'comic_year', 'series_start_year',
                'series_end_year', 'series_duration']]

comic_series.head()

,char_id,comic_id,series_id,comic_name,comic_issue,comic_year,series_start_year,series_end_year,series_duration
0,1009220,43488,16450,A+X,#1,2012.0,2012.0,2014.0,2.0
1,1009220,43501,16450,A+X,#4,2012.0,2012.0,2014.0,2.0
2,1009220,43508,16450,A+X,#9,2012.0,2012.0,2014.0,2.0
3,1009220,17743,3620,A-Next,#2,1998.0,1998.0,1999.0,1.0
4,1009220,17744,3620,A-Next,#3,1998.0,1998.0,1999.0,1.0


__________
Finally, let's join the character data with the merged comic/series data. 

In [239]:
merge_marvel = pd.merge(marvel_df, comic_series, on=['char_id'], how='left')

In [240]:
merge_marvel.head()

,char_id,char_name,char_description,record_mod_date,record_mod_time,comics_total,series_total,stories_total,events_total,url_1,url_2,url_3,comic_id,series_id,comic_name,comic_issue,comic_year,series_start_year,series_end_year,series_duration
0,1009220,Captain America,"Vowing to serve his country any way he could, ...",2016-09-06,11:37:19-0400,2350,643,3491,30,detail: http://marvel.com/comics/characters/10...,wiki: http://marvel.com/universe/Captain_Ameri...,comiclink: http://marvel.com/comics/characters...,43488,16450,A+X,#1,2012.0,2012.0,2014.0,2.0
1,1009220,Captain America,"Vowing to serve his country any way he could, ...",2016-09-06,11:37:19-0400,2350,643,3491,30,detail: http://marvel.com/comics/characters/10...,wiki: http://marvel.com/universe/Captain_Ameri...,comiclink: http://marvel.com/comics/characters...,43501,16450,A+X,#4,2012.0,2012.0,2014.0,2.0
2,1009220,Captain America,"Vowing to serve his country any way he could, ...",2016-09-06,11:37:19-0400,2350,643,3491,30,detail: http://marvel.com/comics/characters/10...,wiki: http://marvel.com/universe/Captain_Ameri...,comiclink: http://marvel.com/comics/characters...,43508,16450,A+X,#9,2012.0,2012.0,2014.0,2.0
3,1009220,Captain America,"Vowing to serve his country any way he could, ...",2016-09-06,11:37:19-0400,2350,643,3491,30,detail: http://marvel.com/comics/characters/10...,wiki: http://marvel.com/universe/Captain_Ameri...,comiclink: http://marvel.com/comics/characters...,17743,3620,A-Next,#2,1998.0,1998.0,1999.0,1.0
4,1009220,Captain America,"Vowing to serve his country any way he could, ...",2016-09-06,11:37:19-0400,2350,643,3491,30,detail: http://marvel.com/comics/characters/10...,wiki: http://marvel.com/universe/Captain_Ameri...,comiclink: http://marvel.com/comics/characters...,17744,3620,A-Next,#3,1998.0,1998.0,1999.0,1.0


We can also do the same with the stories and events data. 

In [243]:
merge_marvel_stories = pd.merge(marvel_df, stories_df, on=['char_id'], how='left')

In [244]:
merge_marvel_stories.head()

,char_id,char_name,char_description,record_mod_date,record_mod_time,comics_total,series_total,stories_total,events_total,url_1,url_2,url_3,story_id,story_name,story_type
0,1009220,Captain America,"Vowing to serve his country any way he could, ...",2016-09-06,11:37:19-0400,2350,643,3491,30,detail: http://marvel.com/comics/characters/10...,wiki: http://marvel.com/universe/Captain_Ameri...,comiclink: http://marvel.com/comics/characters...,670,X-MEN (2004) #186,cover
1,1009220,Captain America,"Vowing to serve his country any way he could, ...",2016-09-06,11:37:19-0400,2350,643,3491,30,detail: http://marvel.com/comics/characters/10...,wiki: http://marvel.com/universe/Captain_Ameri...,comiclink: http://marvel.com/comics/characters...,892,THOR (1998) #81,cover
2,1009220,Captain America,"Vowing to serve his country any way he could, ...",2016-09-06,11:37:19-0400,2350,643,3491,30,detail: http://marvel.com/comics/characters/10...,wiki: http://marvel.com/universe/Captain_Ameri...,comiclink: http://marvel.com/comics/characters...,960,3 of ?,cover
3,1009220,Captain America,"Vowing to serve his country any way he could, ...",2016-09-06,11:37:19-0400,2350,643,3491,30,detail: http://marvel.com/comics/characters/10...,wiki: http://marvel.com/universe/Captain_Ameri...,comiclink: http://marvel.com/comics/characters...,1026,Avengers (1998) #81,cover
4,1009220,Captain America,"Vowing to serve his country any way he could, ...",2016-09-06,11:37:19-0400,2350,643,3491,30,detail: http://marvel.com/comics/characters/10...,wiki: http://marvel.com/universe/Captain_Ameri...,comiclink: http://marvel.com/comics/characters...,1041,Avengers (1998) #502,interiorStory


In [245]:
merge_marvel_events = pd.merge(marvel_df, events_df, on=['char_id'], how='left')

In [246]:
merge_marvel_events.head()

,char_id,char_name,char_description,record_mod_date,record_mod_time,comics_total,series_total,stories_total,events_total,url_1,url_2,url_3,event_id,event_name
0,1009220,Captain America,"Vowing to serve his country any way he could, ...",2016-09-06,11:37:19-0400,2350,643,3491,30,detail: http://marvel.com/comics/characters/10...,wiki: http://marvel.com/universe/Captain_Ameri...,comiclink: http://marvel.com/comics/characters...,116,Acts of Vengeance!
1,1009220,Captain America,"Vowing to serve his country any way he could, ...",2016-09-06,11:37:19-0400,2350,643,3491,30,detail: http://marvel.com/comics/characters/10...,wiki: http://marvel.com/universe/Captain_Ameri...,comiclink: http://marvel.com/comics/characters...,314,Age of Ultron
2,1009220,Captain America,"Vowing to serve his country any way he could, ...",2016-09-06,11:37:19-0400,2350,643,3491,30,detail: http://marvel.com/comics/characters/10...,wiki: http://marvel.com/universe/Captain_Ameri...,comiclink: http://marvel.com/comics/characters...,303,Age of X
3,1009220,Captain America,"Vowing to serve his country any way he could, ...",2016-09-06,11:37:19-0400,2350,643,3491,30,detail: http://marvel.com/comics/characters/10...,wiki: http://marvel.com/universe/Captain_Ameri...,comiclink: http://marvel.com/comics/characters...,231,Armor Wars
4,1009220,Captain America,"Vowing to serve his country any way he could, ...",2016-09-06,11:37:19-0400,2350,643,3491,30,detail: http://marvel.com/comics/characters/10...,wiki: http://marvel.com/universe/Captain_Ameri...,comiclink: http://marvel.com/comics/characters...,234,Avengers Disassembled


-------------
## Conclusion - Final Thoughts.
This was my attempt at navigating the Marvel API. Some things I will like to try next round is to use the extracted Comic, Event, Stories and Series ID's to potentially requery the API and get more information. The API is pretty generous in terms of to how many times calls you can make (3000 as of the time of this writting), however and following the old adage that "only fools rush in", I probablt want to figure out what I want to get out of it. Once I do, there will be a part two for this.  